# 第9章: RNNとCNN
言語処理100本ノック (https://nlp100.github.io/ja/) の第9章です。

## 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [2]:
import pandas as pd

df = pd.read_csv("train.txt", sep="\t", index_col=0, names=["id", "title", "url", "publisher", "category", "story", "hostname", "timestamp"])
df.head()

,title,url,publisher,category,story,hostname,timestamp
id,,,,,,,
NaN,title,url,publisher,category,story,hostname,timestamp
0.0,Taco Bell reveals 'secret' ingredients of myst...,http://www.dailymail.co.uk/news/article-261629...,Daily Mail,b,dqycDHxL4IT2bfM80cii2l60F1oaM,www.dailymail.co.uk,1398870059991
1.0,RPT-UPDATE 2-Carlyle hires JPMorgan's Cavanagh...,http://www.reuters.com/article/2014/03/25/carl...,Reuters,b,d6vEKclqV_iZEsM3pQ3S-2mnaqZUM,www.reuters.com,1395771699595
2.0,Argentina Deposits $1 Billion For June 30 Bond...,http://www.businessweek.com/news/2014-06-26/ar...,Businessweek,b,dAnZOhVBFIp0-qM-n4Cz0cJ6bJWiM,www.businessweek.com,1403853546347
3.0,Banksy art work showing government agents spyi...,http://www.dailymail.co.uk/news/article-260378...,Daily Mail,e,dcBt4tU0L52iiTMQ4rip8YRmnr9xM,www.dailymail.co.uk,1397518333460


In [15]:
import re

def word_cleaning(word):
    word = re.sub("[?.,:;]", "", word)
    return word

print(word_cleaning(";w:,or.?d"))

word


In [17]:
seq_list = list(df["title"])
word_list = []
for s in seq_list[1:]:
    word_list.extend(s.split(" "))
word_list = [word_cleaning(w.lower()) for w in word_list]
word_list = [w for w in word_list if w != ""]
print(word_list[:20])
print(len(word_list))
word_list.count("")

['taco', 'bell', 'reveals', "'secret'", 'ingredients', 'of', 'mystery', 'beef', "that's", '88', 'per', 'cent', 'cow', 'rpt-update', '2-carlyle', 'hires', "jpmorgan's", 'cavanagh', 'as', 'co-president']
109457


0

In [18]:
unique_word_list = list(set(word_list))
print(len(unique_word_list))

17164


In [19]:
word_count_dict = {}
for word in unique_word_list:
    word_count_dict[word] = word_list.count(word)
print(word_count_dict)

{'xl': 1, 'brandon': 1, "'whale": 1, 'dorfman': 8, "policy'": 1, 'peak': 7, 'veg': 1, 'fall': 94, "'too-big-to-fail'": 1, 'jada': 9, 'note': 8, 'holograms': 2, 'cumia': 1, 'jude': 8, "killer'": 1, 'turbines': 1, 'pittsburgh': 1, 'freddie': 16, 'wagner': 1, 'espirito': 3, 'fortify': 1, 'chapter': 4, 'revellers': 2, 'comics': 1, '5-jpmorgan': 1, 'we': 48, 'staying': 1, 'unnatural': 1, 'swiped': 1, 'specter': 2, 'slides': 17, 'paltrow': 39, 'bar': 5, 'ac/dc': 8, 'climbed': 2, 'ignores': 1, 'friend': 9, 'walker': 25, 'assigns': 2, 'acne': 2, 'gumball': 2, 'exchange-traded': 1, 'gul': 1, 'rpt-wall': 1, 'landfill': 1, 'corey': 6, '40m': 2, 'excluded': 1, 'youporn': 1, 'adult': 7, 'simulator': 1, 'appoints': 6, 'surgery': 7, 'evil': 5, '1-cameron': 1, '1-putin': 1, 'klier': 1, 'vitacost': 1, 'review-keaton': 1, 'gloom': 6, 'disputes': 4, 'whilst': 6, 'headed': 6, "dc's": 1, 'cinco': 6, 'buffett': 7, 'prays': 1, 'position': 1, 'peter': 14, "'caused": 1, 'litigation': 6, 'self-portrait': 1, 'fa

In [20]:
word_count_list = []
for key, value in word_count_dict.items():
    word_count_list.append((key, value))
print(word_count_list[:10])

[('xl', 1), ('brandon', 1), ("'whale", 1), ('dorfman', 8), ("policy'", 1), ('peak', 7), ('veg', 1), ('fall', 94), ("'too-big-to-fail'", 1), ('jada', 9)]


In [21]:
word_count_list.sort(key=lambda x: x[1], reverse=True)
print(word_count_list[:10])

[('to', 2868), ('in', 1911), ('the', 1464), ('of', 1423), ('on', 1323), ('for', 1302), ('as', 1138), ('update', 1010), ('-', 971), ('and', 967)]


In [22]:
word_id_dict = {}

for i, v in enumerate(word_count_list):
    word = v[0]
    count = v[1]
    if count > 1:
        word_id_dict[word] = i
    else:
        word_id_dict[word] = 0
print(word_id_dict)

{'to': 0, 'in': 1, 'the': 2, 'of': 3, 'on': 4, 'for': 5, 'as': 6, 'update': 7, '-': 8, 'and': 9, 'us': 10, 'a': 11, 'with': 12, 'at': 13, 'is': 14, 'after': 15, 'new': 16, 'says': 17, 'from': 18, 'up': 19, 'by': 20, 'kardashian': 21, 'over': 22, 'her': 23, 'kim': 24, 'be': 25, 'stocks': 26, 'more': 27, 'first': 28, 'about': 29, 'may': 30, 'data': 31, 'china': 32, 'are': 33, 'shares': 34, 'will': 35, '(1)': 36, 'euro': 37, 'it': 38, 'not': 39, 'deal': 40, 'that': 41, 'sales': 42, 'out': 43, 'profit': 44, 'miley': 45, 'but': 46, 'fed': 47, 'ecb': 48, 'rise': 49, 'global': 50, 'billion': 51, 'you': 52, 'she': 53, 'off': 54, 'his': 55, 'cyrus': 56, 'ceo': 57, 'bank': 58, 'low': 59, 'has': 60, 'was': 61, 'high': 62, 'could': 63, 'growth': 64, 'st': 65, 'west': 66, 'chris': 67, 'day': 68, 'justin': 69, 'million': 70, 'kanye': 71, 'falls': 72, 'ukraine': 73, 'have': 74, 'set': 75, 'buy': 76, 'into': 77, 'bln': 78, 'he': 79, 'this': 80, 'than': 81, 'shows': 82, 'rises': 83, 'gains': 84, 'star'

In [23]:
def word_to_id(word):
    if word in word_id_dict.keys():
        return word_id_dict[word]
    else:
        return 0

def seq_to_id_list(seq):
    word_list = s.split(" ")
    word_list = [word_cleaning(w.lower()) for w in word_list]
    word_list = [w for w in word_list if w != ""]
    return [word_to_id(w) for w in word_list]

print(seq_to_id_list(seq_list[1]))

[4613, 57, 943, 934, 3227, 234, 1209, 33, 1399]
